<a href="https://colab.research.google.com/github/akbarsigit/gpt_from_scratch/blob/main/gpt_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Most of this code is taken from Andrej Karpathy, where I edited some of its content to fits more with my understanding

## Datasets from shakespeare dataset

In [4]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-01 08:27:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-07-01 08:27:37 (47.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [5]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [7]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
# create a mapping from characters to integers
# stoi : char to integers
stoi = { ch:i for i,ch in enumerate(chars) }

# itos : integer to char
itos = { i:ch for i,ch in enumerate(chars) }

stoi

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [10]:
itos

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'A',
 14: 'B',
 15: 'C',
 16: 'D',
 17: 'E',
 18: 'F',
 19: 'G',
 20: 'H',
 21: 'I',
 22: 'J',
 23: 'K',
 24: 'L',
 25: 'M',
 26: 'N',
 27: 'O',
 28: 'P',
 29: 'Q',
 30: 'R',
 31: 'S',
 32: 'T',
 33: 'U',
 34: 'V',
 35: 'W',
 36: 'X',
 37: 'Y',
 38: 'Z',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}

In [11]:
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [12]:
# More readable format for me

def encodeNew(string):
  output = []
  for char in string:
    output.append(stoi[char])
  return output

def decodeNew(array):
  output = []
  for integer in array:
    output.append(itos[integer])
  return "".join(output)

print(encodeNew("hii there"))
print(decodeNew(encodeNew("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [13]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.int64) #torch.int64 or torch.long
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [14]:
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [15]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [16]:
# Get the sampled sentence from the training set

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

# Y is kindda like X but shifted + 1 to the right
# Then we train on the 8 all example from context 1 all the way to the block size not just for eficiency / computational resource
# but to make transformer network be used to seen from as little to just 1 context, all the way to and in between of block size (8) context!

# After block size, we want to truncating all sentencce after it because transformer will never have context greater than the block size inputs!


for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


#### We also need to batched these samples to stacked tensors for efficiency, too keep the GPU busy! (very good at parellel processing)

In [18]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

#   Returns a tensor filled with random integers generated uniformly
#   between low (inclusive) and high (exclusive).The shape of the tensor size.
#   low (int, optional): Lowest integer to be drawn from the distribution. Default: 0.
#   high (int): One above the highest integer to be drawn from the distribution.
#   size (tuple): a tuple defining the shape of the output tensor

    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])

    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y



In [19]:
# so this ix is to get 4 random sequence on the dataset and treat it as an sample
# then the highest possible value to be an sample is length of the data - block size
# because we want max context lenght of 8 => so we dont out of memory!

torch.randint(len(data) - block_size, (batch_size,))

tensor([1078327,  453969,   41646,  671252])

In [20]:
ix = torch.randint(len(data) - block_size, (batch_size,))
print("random seq number", ix)
print(torch.stack([data[i:i+block_size] for i in ix]))

random seq number tensor([1081275,  241627,  748567,  905830])
tensor([[53, 59, 56,  0, 41, 39, 40, 47],
        [63, 53, 59,  1, 39, 50, 50,  6],
        [ 0, 24, 17, 27, 26, 32, 17, 31],
        [ 1, 39, 52, 42,  1, 59, 52, 56]])


In [21]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

inputs:
torch.Size([4, 8])
tensor([[43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39]])
targets:
torch.Size([4, 8])
tensor([[ 1, 51, 39, 63,  1, 40, 43,  1],
        [46, 43,  1, 43, 39, 56, 57, 10],
        [58, 47, 53, 52, 12,  1, 37, 53],
        [56, 43,  1, 21,  1, 41, 39, 51]])
----


In [22]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [43] the target: 1
when input is [43, 1] the target: 51
when input is [43, 1, 51] the target: 39
when input is [43, 1, 51, 39] the target: 63
when input is [43, 1, 51, 39, 63] the target: 1
when input is [43, 1, 51, 39, 63, 1] the target: 40
when input is [43, 1, 51, 39, 63, 1, 40] the target: 43
when input is [43, 1, 51, 39, 63, 1, 40, 43] the target: 1
when input is [58] the target: 46
when input is [58, 46] the target: 43
when input is [58, 46, 43] the target: 1
when input is [58, 46, 43, 1] the target: 43
when input is [58, 46, 43, 1, 43] the target: 39
when input is [58, 46, 43, 1, 43, 39] the target: 56
when input is [58, 46, 43, 1, 43, 39, 56] the target: 57
when input is [58, 46, 43, 1, 43, 39, 56, 57] the target: 10
when input is [39] the target: 58
when input is [39, 58] the target: 47
when input is [39, 58, 47] the target: 53
when input is [39, 58, 47, 53] the target: 52
when input is [39, 58, 47, 53, 52] the target: 12
when input is [39, 58, 47, 53, 52, 12] th

In [23]:
print(xb) # our input to the transformer

tensor([[43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39]])


In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        # print(idx.shape)
        # This is for bigram. I input one character, and trying to predict the next char.
        # so this IDX is the first one char
        for i in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) # this is to call the bigram function, inside the class.
            # print(logits.shape)
            # print(logits)
            # focus only on the last time step
            # print(logits[:, -1, :])
            # from the logits, we want the last character to predict the next word. Because Then IDX will become array of outputs, so we choose the latest output to be the last char.
            logits = logits[:, -1, :] # becomes (B, C)
            # print(logits.shape)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) this will update the array of output. So when we predict K after A, then the idx becomes [A, K] => [A, K, B] etc...
            # print(idx.shape)
            # if i == 2:
            #   break
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.6841, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [25]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [26]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.587916374206543


In [27]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


xiKi-RJ:CgqVuUa!U?qMH.uk!sCuMXvv!CJFfx;LgRyJknOEti.?I&-gPlLyulId?XlaInQ'q,lT$
3Q&sGlvHQ?mqSq-eON
x?SP fUAfCAuCX:bOlgiRQWN:Mphaw
tRLKuYXEaAXxrcq-gCUzeh3w!AcyaylgYWjmJM?Uzw:inaY,:C&OECW:vmGGJAn3onAuMgia!ms$Vb q-gCOcPcUhOnxJGUGSPJWT:.?ujmJFoiNL&A'DxY,prZ?qdT;hoo'dHooXXlxf'WkHK&u3Q?rqUi.kz;?Yx?C&u3Qbfzxlyh'Vl:zyxjKXgC?
lv'QKFiBeviNxO'm!Upm$srm&TqViqiBD3HBP!juEOpmZJyF$Fwfy!PlvWPFC
&WDdP!Ko,px
x
tREOE;AJ.BeXkylOVD3KHp$e?nD,.SFbWWI'ubcL!q-tU;aXmJ&uGXHxJXI&Z!gHRpajj;l.
pTErIBjx;JKIgoCnLGXrJSP!AU-AcbczR?


### Math trick for self attention

In [28]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [29]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

#### a normal intuitive way of doing or "remembering" the contex is by doing weighted average for each of the current and past character. So in here, we do for loop to do that!

In [30]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C) => the b dimension is out, then its just becomes (t, c)
        # then we average the 0th dim, the time/ sequence. => its becomes running means, sorta
        xbow[b,t] = torch.mean(xprev, 0)
print(xbow.shape)

torch.Size([4, 8, 2])


In [31]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [32]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

#### this method of for looping for each dimension and sequence is inefficient, and we can improve better with some math trick using matrix multiplication

In [33]:
# example

a = torch.tril(torch.ones(3, 3)) # this is to make a lower triangular matrix of one.

# so the idea is, to use the nature of lower triangular matrix multiplication, that will sorta do weighted sum over the matrix.
# so we just have to use A matrix as a way to do the for loop, then averaging over a to make the multiplication and summation to be one (weighted)!
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[8., 6.],
        [5., 2.],
        [4., 4.]])
--
c=
tensor([[8.0000, 6.0000],
        [6.5000, 4.0000],
        [5.6667, 4.0000]])


In [34]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)

print(wei.shape, x.shape)

#    8, 8
# 4, 8, 2
# (8, 8)@(4, 8, 2) => (4, 8, 2)
# (8,8)@(8,2)

xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
print(xbow2.shape)

print(xbow[1])
print(xbow2[1])

# this is showing identical, but somehow we get false on the allclose. I suspect there's difference on the floating point number??
print(torch.allclose(xbow[1], xbow2[1]))
torch.allclose(xbow, xbow2)

torch.Size([8, 8]) torch.Size([4, 8, 2])
torch.Size([4, 8, 2])
tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0.0020],
        [ 0.0712, -0.1128],
        [ 0.2527,  0.2149]])
tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0.0020],
        [ 0.0712, -0.1128],
        [ 0.2527,  0.2149]])
False


False

##### a little note about how matrix multiplication are broadcasted in the torch matmul / @ !!!!

In [35]:
# so this is not broadcasted. because the matrix multiplication can do fine
# (3, 4)@(4) --> (3)
tensor1 = torch.randn(3, 4)
tensor2 = torch.randn(4)

(torch.matmul(tensor1, tensor2)).shape
(tensor1 @ tensor2).shape

torch.Size([3])

In [36]:
# this will broadcast or doing matmul to becomes (30, 4)@(4) --> (10 x 3)

tensor1 = torch.randn(10, 3, 4)
tensor2 = torch.randn(       4)
(torch.matmul(tensor1, tensor2)).shape

torch.Size([10, 3])

In [37]:
# this will broadcast or doing matmul to becomes (30, 4)@(4, 10) --> (10 x 3 x 10)

tensor1 = torch.randn(10, 3, 4)
tensor2 = torch.randn(    4,  10)
(torch.matmul(tensor1, tensor2)).shape

torch.Size([10, 3, 10])

In [38]:
# this will broadcast or doing matmul to becomes (2, 8)@(10, 8, 3) --> (10 x 2 x 3)
# (10, 2, 8)
# (10, 8, 3)
# --> (2, 8) @ (8, 3)
tensor1 = torch.randn(    2, 8)
tensor2 = torch.randn(10, 8, 3)
(torch.matmul(tensor1, tensor2)).shape

torch.Size([10, 2, 3])

In [39]:
tensor1 = torch.randn(       8)
tensor2 = torch.randn(10, 8, 3)
(torch.matmul(tensor1, tensor2)).shape

torch.Size([10, 3])

#### another method is to use softmax function

In [40]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
print("tril", tril)
wei = torch.zeros((T,T))
print("wei", wei)
# this will change the 0 on the tril to be inf, and use it as mask on the wei matrix
wei = wei.masked_fill(tril == 0, float('-inf'))
print("wei_masked", wei)
# this will do softmax, which using this formula (e^xi)/sum(e^x). so e^-inf will be 0.
wei = F.softmax(wei, dim=0)
print("wei_softmax", wei)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

# Note: we will end-up using this version because we can set wei as an affinity, or how close word/ char relate to each other on a positional way.
# So the wei here can be treated as weight on how sequence of word related to each other!

tril tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
wei tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
wei_masked tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],


False

In [57]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei.shape)
print(wei[0])
wei = F.softmax(wei, dim=-1)
print(wei[0])
print("wei:", wei.shape)

v = value(x) # (B, T, 16)
out = wei @ v
#out = wei @ x
print("out:", out.shape)
print(out)

torch.Size([4, 8, 8])
tensor([[-1.7629,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-3.3334, -1.6556,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-1.0226, -1.2606,  0.0762,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.7836, -0.8014, -0.3368, -0.8496,    -inf,    -inf,    -inf,    -inf],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,    -inf,    -inf,    -inf],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,    -inf,    -inf],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,    -inf],
        [-1.8044, -0.4126, -0.8306,  0.5899, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

### Doing scaled dot product attention is important when calculating wei (Q and K.T) because although the k and q is in gaussian distribution with variance of unit 1,  when we do dot product naively, the wei matrix will have variance of ~16 (head_size), and thats bad!. so we do normalization on the wei matrix by additional divides of the sqrt head_size!!!!

In [42]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) #* head_size**-0.5 ==> with no normalization, this will produce high variance matrix

In [43]:
k.var()

tensor(1.0449)

In [44]:
q.var()

tensor(1.0700)

In [45]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [46]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [47]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [48]:
wei.var() # Resulting with ~16 varianceeeee

tensor(17.4690)

In [49]:
# but when we normalize with sqrt of the head size, we got much better variance of roughly unit (1)
wei = q @ k.transpose(-2, -1) * head_size**-0.5
wei.var()

tensor(1.0918)

### NOTE: why doing normalization of wei with variance of unit (1) is important? Because when we want to input our wei into the softmax,


> wei = F.softmax(wei, dim=-1)

### we want the wei to be normal, especially on initialization that our matrix is normal, because we want the model to be fairly deffuse (have much value to be around 0) so that the model can actually learn.

#### we dont want at initialization to have value that is extreme because the nature of softmax will give heavy weight on the exteme value that is not nessecary right (false confident), and the prediction is is just considering one node and not considering other nodes to make good prediction. So we dont want that to happend!!




In [50]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1) # this is fairlly deffuse input to the softmax, with many value close to each other at initialization

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [51]:
# we scaled it to be 8 times the value (extreme value), and the softmax results with dominant value, and this will make the nodes to just considering the greatest value
# and discard the others!
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot 1/0

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

#### Batch norm is like this layer norm, but layer norm is just mean along the 1st dimention (row), instead of batch norm that is along the collumn.

In [52]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    # (3, 4) ==> x.mean(0, keepdim=true) ==> (1, 4) ==> batch norm
    # (3, 4) ==> x.mean(1, keepdim=true) ==> (3, 1) ==> layer norm
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

print(x)

tensor([[ 0.1335, -0.1059, -0.3824,  ..., -1.3422, -0.1971,  0.8795],
        [-0.0353, -0.7439, -0.3371,  ..., -0.6276, -0.4846,  0.4556],
        [ 0.3069, -1.5010,  1.4898,  ..., -0.6819,  0.9993,  0.8382],
        ...,
        [-1.6080, -1.6324, -0.7634,  ..., -0.9847,  0.0039, -0.8610],
        [-0.2273,  0.0066, -0.2763,  ..., -0.8705, -1.2442, -0.7531],
        [ 0.3054, -0.1505, -0.3809,  ..., -1.4962, -0.7711, -1.0681]])


In [53]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs
# ===> if mean 0 and std 1 across all batch is called batch norm.

(tensor(0.1469), tensor(0.8803))

In [54]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features
# ===> but if mean 0 and std 1 across all features is called layer norm

(tensor(-9.5367e-09), tensor(1.0000))